In [1]:
import os
from scripts.vllm_server import VLLMServer
import dspy
from dspy.adapters.types import Image
from dspy.teleprompt import BootstrapFewShot, LabeledFewShot
import json
from typing import Optional
from pathlib import Path
import random
from sklearn.metrics import mean_absolute_error


random.seed(42)

In [ ]:
AVAILABLE_MODELS = ["Qwen2.5-VL-7B-Instruct", "gemma-3-27b-it"]


MODEL_NAME = AVAILABLE_MODELS[0]
assert MODEL_NAME in AVAILABLE_MODELS

In [3]:
config_path = os.path.abspath(f"configs/{MODEL_NAME}.yaml")

In [4]:
server = VLLMServer(config_path)

Не удалось убить старый процесс: [Errno 3] No such process
🚀 Запуск vLLM-сервера на localhost:1337 с моделью /home/student/vllm_models/Qwen/Qwen2.5-VL-7B-Instruct…
⏳ Ожидание порта 1337 на localhost…


✅ Порт 1337 на localhost открыт — сервер готов.
📜 Логи сервера пишутся в файл: vllm_server.log


In [5]:
lm = dspy.LM(
    f"openai/{MODEL_NAME}",
    api_base="http://localhost:1337/v1",
    api_key="local",
    model_type="chat",
    max_tokens=4096,
)
dspy.configure(lm=lm)

In [6]:
lm(prompt="Привет! Как твои дела?")

/home/student/kirienko/.venv/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Прив...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...s={'stop_reason': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


['Привет! Я искусственный интеллект, поэтому у меня нет чувств или эмоций, но я всегда готов помочь вам с любыми вопросами или задачами, которые вы можете иметь. Как я могу вам помочь сегодня?']

/home/student/kirienko/.venv/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## en...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...s={'stop_reason': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/home/student/kirienko/.venv/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## en...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - seria

In [38]:
AVAILABLE_OPTIMIZERS = ["bs_few_shot", "few_shot"]

OPTIMIZER = AVAILABLE_OPTIMIZERS[0]
FEW_SHOTS_COUNT = 30
TRAIN_SAMPLES_CNT = 1000
TEST_SAMPLES_CNT = 500
IMAGES_DIR_PATH = "data/test_valid"
DATA_FILE_PATH = "data/test_fix.json"
EXP_NAME = f"{MODEL_NAME}_{OPTIMIZER}_{FEW_SHOTS_COUNT}"
ARTIFACTS_DIR = "vlm_experiments"

assert OPTIMIZER in AVAILABLE_OPTIMIZERS

In [39]:
class NutritionSignature(dspy.Signature):
    """
    You are a nutrition expert.  
    Your task is to analyze an image of a dish and estimate its nutritional value per 100 grams: calories, protein, fat, and carbohydrates.

    If the name of the dish is provided, use it together with the image to make a more accurate assessment.
    """
    image: Optional[Image] = dspy.InputField(desc="Dish image")
    title: str = dspy.InputField(desc="Dish title")

    energy_kcal: float = dspy.OutputField(desc="Calories per 100g")
    protein_g: float = dspy.OutputField(desc="Protein per 100g")
    fat_g: float = dspy.OutputField(desc="Fat per 100g")
    carbs_g: float = dspy.OutputField(desc="Carbohydrates per 100g")

In [40]:
def prepare_example(item: dict, images_dir_path: str) -> dspy.Example:
    for image in item.get("images"):
        try:
            image_path = Path(images_dir_path) / Path(image["valid_path"].replace('\\', '/'))
            image = Image.from_file(str(image_path))
        except Exception as e:
            print(str(e))
            continue

    nutr100 = item["nutr_per100g"]

    return dspy.Example(
        image=image,
        title=item["title"],
        energy_kcal=nutr100["energy_kcal"],
        protein_g=nutr100["protein_g"],
        fat_g=nutr100["fat_g"],
        carbs_g=nutr100["carbs_g"]
    ).with_inputs("image", "title")

In [10]:
with open(DATA_FILE_PATH, "r") as f:
    data = json.load(f)

random.shuffle(data)

train_examples = [prepare_example(x, IMAGES_DIR_PATH) for x in data[:TRAIN_SAMPLES_CNT]]
test_examples = [prepare_example(x, IMAGES_DIR_PATH) for x in data[TRAIN_SAMPLES_CNT:TRAIN_SAMPLES_CNT + TEST_SAMPLES_CNT]]

In [11]:
train_examples[0]

Example({'image': Image(url=data:image/jpeg;base64,<IMAGE_BASE_64_ENCODED(100176)>), 'title': 'Simple Spicy Potato Salad', 'energy_kcal': 120.61419586191194, 'protein_g': 3.2724255058769955, 'fat_g': 6.327658878105701, 'carbs_g': 12.643890983863162}) (input_keys={'title', 'image'})

In [41]:
nutrition_predictor = dspy.ChainOfThought(NutritionSignature)

In [42]:
def mean_negative_mae(example: dspy.Example, prediction: dspy.Prediction, trace=None) -> float:
    energy_kcal_mae = abs(example.energy_kcal - prediction.energy_kcal)
    protein_g_mae = abs(example.protein_g - prediction.protein_g)
    fat_g_mae = abs(example.fat_g - prediction.fat_g)
    carbs_g_mae = abs(example.carbs_g - prediction.carbs_g)

    return -(energy_kcal_mae + protein_g_mae + fat_g_mae + carbs_g_mae) / 4


evaluator = dspy.Evaluate(
    devset=test_examples,
    metric=mean_negative_mae,
    display_progress=True,
    display_table=True,
    failure_score=-1000
)

In [43]:
max_labeled_demos = int(FEW_SHOTS_COUNT * 0.8)
max_bootstrapped_demos = FEW_SHOTS_COUNT - max_labeled_demos

assert max_bootstrapped_demos + max_labeled_demos == FEW_SHOTS_COUNT

In [44]:
if OPTIMIZER == "bs_few_shot":
    optimizer = BootstrapFewShot(
        metric=mean_negative_mae,
        max_bootstrapped_demos=max_bootstrapped_demos,
        max_labeled_demos=max_labeled_demos, 
        max_errors=len(train_examples)
    )
elif OPTIMIZER == "few_shot":
    optimizer = LabeledFewShot(k=FEW_SHOTS_COUNT)
else:
    raise ValueError("Invalid optimizer") 

In [45]:
optimized_nutrition_predictor = optimizer.compile(
    nutrition_predictor,
    trainset=train_examples,
)

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/student/kirienko/.venv/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...s={'stop_reason': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
  1%|          | 6/1000 [00:48<2:14:39,  8.13s/it]

Bootstrapped 6 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.


In [17]:
evaluator(optimized_nutrition_predictor)

Average Metric: -24590.07 / 500 (-4918.0%): 100%|██████████| 500/500 [08:48<00:00,  1.06s/it]

2025/06/17 19:26:23 INFO dspy.evaluate.evaluate: Average Metric: -24590.070253186408 / 500 (-4918.0%)


,image,title,example_energy_kcal,example_protein_g,example_fat_g,example_carbs_g,pred_energy_kcal,pred_protein_g,pred_fat_g,pred_carbs_g,mean_negative_mae
0,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Butter Naan (Indian Bread),360.009374,10.127715,0.960809,77.712808,100.0,3.0,5.0,15.0,✔️ [-83.472]
1,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Drunken Berries,98.209099,0.648934,0.264089,23.309142,100.0,1.0,0.5,15.0,✔️ [-2.672]
2,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Cocktail Nuts,507.090303,11.577786,48.301195,6.517101,600.0,10.0,45.0,0.0,✔️ [-26.076]
3,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Steamed Fish with Ginger,204.405281,12.831889,15.945791,2.391402,150.0,20.0,2.0,1.0,✔️ [-19.228]
4,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",5 minute Pizza base. No yeast!,211.500000,10.260000,0.685000,41.073750,250.0,10.0,15.0,30.0,✔️ [-16.037]
...,...,...,...,...,...,...,...,...,...,...,...
495,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Roasted Garlic-Pepper Green Beans,72.755273,3.081118,1.844977,10.956502,100.0,2.0,2.0,5.0,✔️ [-8.609]
496,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Mustard Dip,251.235101,2.314217,25.244256,3.694982,100.0,2.0,8.0,2.0,✔️ [-42.622]
497,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Hot Bacon Dressing,273.939227,7.189694,4.622560,50.894353,120.0,2.0,10.0,2.0,✔️ [-53.350]
498,"<<CUSTOM-TYPE-START-IDENTIFIER>>[{'type': 'image_url', 'image_url'...",Healthy Banana Drop Cookies,270.860268,4.611757,1.960738,58.691651,250.0,2.0,1.0,30.0,✔️ [-13.281]


-4918.01

In [46]:
preds = optimized_nutrition_predictor.batch([x.inputs() for x in test_examples])

Processed 6 / 500 examples:   1%|          | 6/500 [00:24<14:00,  1.70s/it]  

/home/student/kirienko/.venv/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="[[ ## re...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...s={'stop_reason': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Processed 403 / 500 examples:  81%|████████  | 403/500 [12:29<02:56,  1.82s/it]

2025/06/17 20:02:08 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Processed 409 / 500 examples:  82%|████████▏ | 408/500 [12:45<04:27,  2.91s/it]

/home/student/kirienko/.venv/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='{\n  "re...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...s={'stop_reason': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Processed 500 / 500 examples: 100%|██████████| 500/500 [15:30<00:00,  1.86s/it]


In [47]:
nutrients = ['energy_kcal', 'protein_g', 'fat_g', 'carbs_g']

targets = []
predictions = []

for pred, example in zip(preds, test_examples):
    target = {k: v for k, v in example.items() if k in nutrients}
    targets.append(target)
    predictions.append(pred.toDict())

In [48]:
y_true = {nutrient: [t[nutrient] for t in targets] for nutrient in nutrients}
y_pred = {nutrient: [p[nutrient] for p in predictions] for nutrient in nutrients}

mae_scores = {}

for nutrient in nutrients:
    mae = mean_absolute_error(y_true[nutrient], y_pred[nutrient])
    mae_scores[nutrient] = mae

In [49]:
os.makedirs(f"{ARTIFACTS_DIR}/{EXP_NAME}", exist_ok=True)

with open(f"{ARTIFACTS_DIR}/{EXP_NAME}/metrics.json", "w", encoding="utf-8") as f:
    json.dump(mae_scores, f, indent=4, ensure_ascii=False)

with open(f"{ARTIFACTS_DIR}/{EXP_NAME}/params.json", "w", encoding="utf-8") as f:
    params = {
        "model": MODEL_NAME,
        "optimizer": OPTIMIZER,
        "few_shots_count": FEW_SHOTS_COUNT,
        "train_samples_cnt": TRAIN_SAMPLES_CNT,
        "test_samples_cnt": TEST_SAMPLES_CNT
    }
    json.dump(params, f, indent=4, ensure_ascii=False)

optimized_nutrition_predictor.save(f"{ARTIFACTS_DIR}/{EXP_NAME}/dspy_program.json")

In [23]:
server.stop()